# Projet TextRank
**Avant toute chose, assurez vous de copier ce notebook et de travailler uniquement sur votre copie personnelle**

Préambule
---------

Ce préambule télécharge le dictionnaire word2vec qui associe des mots (`str`) à des vecteurs (`np.array `de taille 200)
Il télécharge aussi le corpus de travail pour ce TP qui est issu de wikipedia



In [ ]:
#INIT notebook
!pip install gensim

import os
from gensim.models import KeyedVectors
from urllib.request import urlretrieve

#Download word vectors if needed (this may take time)
if 'frw2v.bin' not in os.listdir():
  urlretrieve('http://embeddings.net/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin','frw2v.bin')
w2v = KeyedVectors.load_word2vec_format("frw2v.bin",binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#Téléchargement du corpus wikipedia
!pip install wikipedia
import wikipedia

wikipedia.set_lang('fr')

writers = ['Corneille',"Racine",'Hugo','Flaubert','Balzac','Zola','Baudelaire','Rimbaud','Verlaine']
corpus  = [ wikipedia.summary(auteur) for auteur in writers ]


A ce stade, la liste `corpus` contient les textes, le dictionnaire `w2v` associe des mots à des vecteurs de taille 100.

Premier exercice: segmentation en phrases et en mots
---------------

Définir une fonction qui découpe un texte en phrases et qui renvoie une liste de phrases.
Définir une autre fonction qui découpe une phrase en mots. Celle-ci renvoie une liste de mots

In [ ]:
def text_tokenize(text):
  text = text.replace("\n", "")
  return text.split(".")

def sent_tokenize(sentence):
  sentence = cleaning(sentence)
  return sentence.split(" ")

def cleaning(sentence) :
  replace_punctuation = sentence.maketrans(str.punctuation, ' '*len(str.punctuation))
  return sentence.translate(replace_punctuation)

Second exercice: calculer une représentation vectorielle de la phrase par sac de mots
--------------------

Définir une fonction qui prend en argument une liste de mots et qui renvoie un vecteur. Faire attention à gérer le problèmes des mots inconnus du dictionnaire. Faut-il inclure tous les mots pour obtenir un vecteur de phrase informatif ?



Certains mots inconnus doivent être inclus dans les vecteurs ; en effet, les noms propres sont importants pour les résumés. Cependant, tous les mots ne sont pas utiles pour faire un vecteur informatif : les mots grammaticaux ne donnent pas d'information sémantique et sont trop présents pour être représentatifs. Faute d'autre moyen, j'ai choisi de ne prendre en compte que les mots d'au moins quatre lettres. En effet, les mots grammaticaux ont tendance à être plus courts que les mots informatifs, et après plusieurs essais, quatre me semble être la meilleure limite. J'ajoute également les mots contenant des chiffres (nécessaire pour les années, âges, etc.).

In [ ]:
# Je choisis ici de ne pas prendre en compte les mots de 3 lettres ou moins
# (Peu de chance qu'ils soient informatifs)
# J'inclue cependant les mots inconnus (nécessaire pour les noms) ainsi que les nombres
def vectorize_sent(token_list):
  vec_texte = np.zeros(200)
  for tok in token_list :
    if tok in w2v and len(tok) > 3 :
      vec_texte += w2v[tok]
    if tok not in w2v :
      if len(tok) > 2 or bool(re.search(r'\d', tok)) :
        vec_texte += np.zeros(200)
  return vec_texte

Troisième exercice : Définir une fonction qui calcule les similarités entre les phrases
--------------
Définir une fonction qui, à partir d'une liste de phrases, calcule une matrice stochastique construite à partir des similarités cosinus entre chaque couple de phrases. Faut-il réaliser un lissage comme dans le cas du pagerank traditionnel ? pourquoi ?



In [ ]:
def cos_similarity (vectorA, vectorB) :
    prodScal = np.dot(vectorA, vectorB)
    multNorm = norm(vectorA)*norm(vectorB)
    if multNorm != 0 :
      return (prodScal / multNorm)
    return -1

def stoc_matrix (matrix, n) :
  new_matrix = np.copy(matrix)
  for i in range (0, n) :
    somme = 0
    for j in range (0, n) :
      somme += matrix[i, j]
    if somme != 0 :
      for j in range (0, n) :
        new_matrix[i, j] = matrix[i, j] / somme
  return new_matrix


def make_matrix(vector_list,delta):
  n = len(vector_list)
  matrix = np.zeros((n,n))
  for i in range (0, n) :
    for j in range (0, n) :
      matrix[i, j] = cos_similarity(vector_list[i], vector_list[j])
  matrix = stoc_matrix(matrix, n)
  return matrix

Quatrième exercice : calcul du pageRank
-------------
Définir une fonction qui calcule le pageRank de la matrice stochastique et qui sélectionne les K phrases les plus pertinentes d'un texte

In [ ]:
import numpy.random as rand
import heapq
import networkx as nx

def n_max(tab, n) :
  maximums = []
  for _ in range(n) :
    maxi = 0
    for i in range(len(tab)) :
      if tab[i] > tab[maxi] and i not in maximums:
        maxi = i
    maximums.append(maxi)
    tab[maxi] = float('-inf')
  return maximums
  

def ergodique(S,delta) :
  return delta * S + (1-delta) * 1/len(S)

def pagerank(G,K=5):
  x = G[0,:]
  for _ in range(100) :
    x = G @ x
  return n_max(x, K)


Cinquième exercice : Synthese et commentaires
-------------
Rassemblez les solutions obtenues dans les exercices précédents pour afficher le résumé extractif de chacun des textes du corpus

Le paramètre delta choisi pour rendre la méthode ergodique modifie les résultats obtenus ; 0.1 est la meilleure valeur que j'aie trouvée. Cependant, le lissage pour créer la matrice n'est pas nécessaire.

In [ ]:
import string as str
import numpy as np
import re
from numpy.linalg import norm

def summary(texte):
  sentences = text_tokenize(text)
  vector_list = []
  for sentence in sentences :
    token_list = sent_tokenize(sentence)
    vector = vectorize_sent(token_list)
    vector_list.append(vector)
  matrix = make_matrix(vector_list, 0)
  g = ergodique(matrix, 0.1)
  pr = pagerank(g)
  print(pr)
  for indice in pr :
    print(sentences[indice])
  print()

for text in corpus :
  summary(text)

[5, 0, 4, 6, 7]
 Plusieurs de ses confrères, constatant à leur tour que la Fronde avait occasionné un rejet de la tragédie historique et politique, renoncèrent de même à écrire des tragédies ou se concentrèrent sur le genre de la comédie
Pierre Corneille, aussi appelé « le Grand Corneille » ou « Corneille l'aîné », né le 6 juin 1606 à Rouen et mort le 1er octobre 1684 à Paris (paroisse Saint-Roch), est un dramaturge et poète français du XVIIe siècle
Déçu par l'accueil rencontré par Pertharite (1652, pendant les troubles de la Fronde), au moment où le début de sa traduction de L'Imitation de Jésus-Christ connaissait un extraordinaire succès de librairie, il décida de renoncer à l'écriture théâtrale et acheva progressivement la traduction de L'Imitation
 Tenté dès 1656 de revenir au théâtre par le biais d'une tragédie à grand spectacle que lui avait commandée un noble normand (La Conquête de la Toison d'or, créée à Paris six ans plus tard fut l'un des plus grands succès du siècle), occup

Analysez votre démarche et indiquez quels paramètres ont un impact sur le résumé produit au final, comme par exemple:
*  Segmentation et mots inconnus
*  Quels mots inclure dans la vectorisation de la phrase ?
*  Paramètre delta de lissage 
* ... (autres)

A votre avis, comment pourrait-on mesurer la qualité d'un résumé ?

La méthode proposée est-elle convaincante ? A votre avis, comment pourrait-on l'améliorer ?  

Pour mesurer la qualité d'un résumé, on pourrait vérifier que l'on a bien les informations importantes du texte : en l'occurrence, noms, dates importantes, faits marquants de l'auteur.